In [2]:

import os
import boto3
import re
import pandas as pd
import numpy as np
import json
import shutil
import cv2
import os
from IPython.display import clear_output
from sagemaker import get_execution_role
import sys

s3 = boto3.client('s3')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from preprocess import load_data, crop_imgs, save_new_images, preprocess_imgs, split_load_crop

clear_output()

role = get_execution_role()

bucket = 'classification-train-data-0009-6572-9629'# enter your s3 bucket where you will copy data and model artifacts
prefix = 'brain-tumor/dataset'
dataset_file = 'brain-mri-images-for-brain-tumor-detection.zip'

!rm -rf 'dataset' TRAIN TEST VAL TRAIN_CROP TEST_CROP VAL_CROP keras-pretrained
!mkdir 'dataset' TRAIN TEST VAL TRAIN/YES TRAIN/NO TEST/YES TEST/NO VAL/YES VAL/NO
!rm -rf TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO
!mkdir TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO


s3.download_file(bucket, os.path.join(prefix, dataset_file), os.path.join('dataset', dataset_file))

!unzip 'dataset/brain-mri-images-for-brain-tumor-detection.zip' -d 'dataset/'
!rm 'dataset/brain-mri-images-for-brain-tumor-detection.zip'

IMG_PATH = './dataset/brain_tumor_dataset/'
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224,224)

split_load_crop(IMG_PATH, TRAIN_DIR, TEST_DIR, VAL_DIR, IMG_SIZE)


Archive:  dataset/brain-mri-images-for-brain-tumor-detection.zip
   creating: dataset/brain_tumor_dataset/
   creating: dataset/brain_tumor_dataset/no/
  inflating: dataset/brain_tumor_dataset/no/30 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/22 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/41 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/14 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/no 10.jpg  
  inflating: dataset/brain_tumor_dataset/no/18 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/no 9.png  
  inflating: dataset/brain_tumor_dataset/no/no 7.jpeg  
  inflating: dataset/brain_tumor_dataset/no/no 8.jpg  
  inflating: dataset/brain_tumor_dataset/no/no.jpg  
  inflating: dataset/brain_tumor_dataset/no/3 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/43 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/20 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/32 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/2 no.jpeg  
  inflating: d

100%|██████████| 2/2 [00:00<00:00, 25.77it/s]


193 images loaded from TRAIN/ directory.
10 images loaded from TEST/ directory.
50 images loaded from VAL/ directory.
